In [14]:
import logging
import os
import torch
import transformers
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig

In [ ]:
dataset ={'train':[], 'test':[], 'dev':[]}

In [ ]:
def convert_to_dataset(filename, stage, examples):
  idx = 0
  codefile = os.path.join(filename, stage + ".code")
  quesfile = os.path.join(filename, stage + ".question")
  ansfile = os.path.join(filename, stage + ".answer")
  with open(codefile,encoding="utf-8") as code_f:
      with open(ansfile, encoding="utf-8") as ans_f:
          with open(quesfile, encoding="utf-8") as ques_f:
              for codeline, quesline, ansline in zip(code_f,ques_f,ans_f):
                  code = codeline.strip()
                  question = quesline.strip()
                  ans = ansline.strip()
                  temp_list = {'code':code,'questions':question,'answer':ans}
                  examples[stage].append(temp_list)

In [ ]:
convert_to_dataset('/train', 'train', dataset)
convert_to_dataset('/valid', 'dev', dataset)
convert_to_dataset('/test', 'test', dataset)

In [ ]:
json_object = json.dumps(dataset)
with open("codeQA.json", "w") as outfile:
    outfile.write(json_object)